## Week 6 : Large Language Models
```
- Generative Artificial Intelligence (Fall semester 2023)
- Professor: Muhammad Fahim
- Teaching Assistant: Ahmad Taha
```
<hr>

## Contents
```
1. Transformers (Implementing a transformer)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding

```

<hr>


# Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)


In [1]:
import torch
from torch import nn
import torch.optim as optim
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Transformer Encoder with Pytorch

In [2]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=32)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [3]:
encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

## Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
**The main goal is to efficiently encode the data**

![](http://jalammar.github.io/images/t/encoder_with_tensors.png)

## Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They’re abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [4]:
# simple sequence = I am here today
simple_sequence_embedding = torch.rand((4, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))

# Create key, query and value for each word in the senetence
queries = simple_sequence_embedding.mm(Wq) # self.q(embedding[0])
keys = simple_sequence_embedding.mm(Wk)
values = simple_sequence_embedding.mm(Wv)

In [5]:
values

tensor([[-3.2836,  2.6417,  1.8303,  3.2429, -2.6951, -0.8692,  0.8303],
        [-3.0937,  3.0658,  4.8285,  1.7669, -1.5430,  1.2688,  0.6740],
        [-0.4102,  2.6807,  2.2094,  3.0515, -2.3518,  2.2580,  1.5570],
        [-5.0672,  3.1323,  4.6400,  4.8453, -3.1108, -0.2685,  0.7331]])

In [6]:
simple_sequence_embedding

tensor([[0.1351, 0.3158, 0.7930,  ..., 0.3949, 0.9916, 0.9226],
        [0.2482, 0.9978, 0.6790,  ..., 0.8725, 0.3227, 0.7487],
        [0.8038, 0.6503, 0.4323,  ..., 0.2089, 0.3981, 0.4299],
        [0.1158, 0.1166, 0.6750,  ..., 0.1751, 0.7832, 0.5947]])

## Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [7]:
scores = torch.mm(queries, keys.T)
scores

tensor([[  6.9156,   9.8214,  17.7234,   7.7308],
        [ -7.8654,  -5.3478,  -2.6943, -11.4905],
        [-14.3517, -14.2825, -14.4285, -19.9947],
        [ 10.8279,  11.5883,  18.1052,   7.7855]])

## Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [8]:
scores = scores / 8
scores

tensor([[ 0.8644,  1.2277,  2.2154,  0.9663],
        [-0.9832, -0.6685, -0.3368, -1.4363],
        [-1.7940, -1.7853, -1.8036, -2.4993],
        [ 1.3535,  1.4485,  2.2632,  0.9732]])

## Step 4

Pass the result through a softmax operation

In [9]:
scores = torch.nn.functional.softmax(scores, dim=1)
scores

tensor([[0.1350, 0.1941, 0.5213, 0.1495],
        [0.2035, 0.2788, 0.3884, 0.1293],
        [0.2863, 0.2888, 0.2835, 0.1414],
        [0.1899, 0.2088, 0.4715, 0.1298]])

## Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [10]:
scores.shape, values.shape

(torch.Size([4, 4]), torch.Size([4, 7]))

In [11]:
z = scores @ values
z

tensor([[-2.0154,  2.8177,  3.0301,  3.0961, -2.3546,  1.2660,  1.1643],
        [-2.3454,  2.8385,  3.1768,  2.9644, -2.2944,  1.0191,  1.0564],
        [-2.6662,  2.8446,  3.2009,  2.9890, -2.3238,  0.7199,  0.9775],
        [-2.1206,  2.8123,  2.9998,  3.0524, -2.3466,  1.1298,  1.1277]])

# Multi-headed attention

**GOAL**:
1. Expand the model’s ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrices down into a single matrix by concatinating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [12]:
from torch import Tensor
import torch.nn.functional as f


def scaled_dot_product_attention(query, key, value):
  temp = query.bmm(key.transpose(1, 2))
  scale = query.size(-1) ** 0.5
  softmax = f.softmax(temp / scale, dim=-1)
  return softmax.bmm(value)

## Now lets make attention head

In [13]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()

        # Fill in the missing parts of the constructor to initialize the query, key, and value linear transformations
        # dim_in is the input dimension, dim_q and dim_k are the output dimensions for the queries and keys/values respectively
        # Example: dim_in 512, dim_q = dim_k 64 in the paper

        self.query=nn.Linear(dim_in, dim_q)
        self.key = nn.Linear(dim_in, dim_k)
        self.value = nn.Linear(dim_in, dim_k)

    def forward(self, query, key, value):
        # Implement the forward pass by calling scaled_dot_product_attention function
        # You need to transform the query, key, and value using the linear transformations defined in __init__
        # Fill in with the correct method calls
        query=self.query(query)
        key=self.key(key)
        value=self.value(value)

        attention_output=scaled_dot_product_attention(query,key,value)

        return attention_output

## Multi Head Attention

In [43]:
class MultiHeadToAttention(nn.Module):
    def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
        super().__init__()
        # Initialize heads as multi-AttentionHead instances
        # Initialize linear to combine the outputs of all heads into a single output
        # self.heads=nn.Linear(number_of_heads*dim_k, dim_in)
        self.heads = nn.ModuleList([AttentionHead(dim_in, dim_q, dim_k) for _ in range(number_of_heads)])
        self.linear = nn.Linear(number_of_heads * dim_k, dim_in)

    def forward(self, query: Tensor, key: Tensor, value: Tensor):
        # Concatenate outputs from all heads and apply the final linear transformation
        heads_output=[head(query,key,value)  for head in self.heads]
        combined_output=torch.cat(heads_output,dim=-1)

        z=self.linear(combined_output)
        return z

## Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*C3a9RL6-SFC6fW8NGpJg5A.png)

In [44]:
def position_encoding(seq_len, dim_model, device):
    # Define the position tensor 'pos' with dimensions appropriate for sequence length
    pos = torch.arange(seq_len, dtype=torch.float).unsqueeze(1).to(device)


    # Define the dimension tensor 'dim' suitable for the model dimensions
    dim = torch.arange(0, dim_model, step=2, dtype=torch.float).to(device)

    # Calculate the phase using the position and dimension tensors
    phase = pos / (10000 ** (dim / dim_model))

    # Return the sinusoidal position encoding

    pos_encoding=torch.zeros((seq_len,dim_model), device=device)
    pos_encoding[:,0::2]=torch.sin(phase)
    pos_encoding[:,1::2]=torch.cos(phase)


    return  pos_encoding # Complete this line to select sin or cos based on even/odd index

## Encoder Feed Forward

In [45]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
  return nn.Sequential(nn.Linear(dim_input, dim_feedforward),
                       nn.ReLU(),
                       nn.Linear(dim_feedforward, dim_input)
                       )

## Encoder Residual

From the original paper the author implementation

In [46]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

## Putting all together on encoder side

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## Putting the Encoder layer together

In [47]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Define dimensions for query and key based on model dimension and number of heads

        dim_q = dim_k = dim_model // num_heads

        # Initialize the MultiHeadAttention component with a residual connection and dropout
        self.attention = Residual(MultiHeadToAttention(num_heads,dim_model,dim_q,dim_k),dimension=dim_model, dropout=dropout)

        # Initialize the feedforward component with a residual connection and dropout
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dim_model, dropout)

    def forward(self, src):
        # Apply the attention mechanism to the source data
        src = self.attention(src, src, src)

        # Apply the feedforward network to the output of the attention mechanism
        return self.feed_forward(src)

## Putting together transfomer Encoder part

In [48]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers=12, dim_model=512, num_heads=4, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Initialize a list of TransformerEncoderLayer instances
        self.layers = nn.ModuleList([TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers)])

        self.dim_model=dim_model
        self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, src):
        # Retrieve the sequence length and dimension from the source tensor
        seq_len, dimension = src.shape[1], src.shape[2]

        # Add position encoding to the source tensor
        src += position_encoding(seq_len, dimension, device)

        # Process each layer in the transformer encoder

        for layer in self.layers:
            src = layer(src)
        return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.


![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

## Decoder layer

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

**Task**: implement the decoder layer

In [49]:
class TransformerDecoderLayer(nn.Module):
    def __init__(
        self,
        dim_model: int = 512,
        num_heads: int = 6,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # init dim_q and dim_k as in the encoder
        # Initialize the first self-attention layer with a residual connection

        dim_q=dim_k=dim_model // num_heads

        self.attention_1 = Residual(MultiHeadToAttention(num_heads, dim_model, dim_model // num_heads, dim_model // num_heads), dim_model, dropout)

        # Initialize the second attention layer for interaction with the encoder output
        self.attention_2 = Residual(MultiHeadToAttention(num_heads, dim_model, dim_model // num_heads, dim_model // num_heads), dim_model, dropout)

        # Initialize the feed-forward network
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dim_model, dropout)


    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Self-attention mechanism
        tgt = self.attention_1(tgt, tgt, tgt)

        # Cross-attention mechanism where the decoder attends to the encoder's output
        tgt = self.attention_2(tgt, memory, memory)

        # Pass through the feed-forward network
        return self.feed_forward(tgt)

## Full Transfomer Decoder

**Task**: implement the transfomer decoder part class

In [50]:
class TransformerDecoder(nn.Module):
    def __init__(
        self,
        num_layers: int = 6,
        dim_model: int = 512,
        num_heads: int = 8,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # Initialize laters from TransformerDecoderLayer instances
        self.layers = nn.ModuleList([TransformerDecoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers)])

        self.dim_model=dim_model
        self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # Define the output linear transformation
        self.linear = nn.Linear(dim_model,dim_model)

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Calculate sequence length and dimension from the target tensor
        seq_len, dimension = tgt.shape[1], tgt.shape[2]

        # Add position encoding to the target tensor
        tgt += position_encoding(seq_len, dimension, device)

        # Process each layer in the transformer decoder
        # Code here
        for layer in self.layers:
            tgt = layer(tgt, memory)

        # Apply a softmax to the output of the final linear layer
        tgt = self.linear(tgt)
        tgt = f.softmax(tgt, dim=2)
        return tgt

## Full Transfomer model

**Task**:
1. Assembly a full transfomer (Encoder + Decoder)
2. Implement the Transfomer training loop
3. Using dataset of your choice train the transformer just for one epoch


In [51]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

class Transformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers: int = 6,
        num_decoder_layers: int = 6,
        dim_model: int = 512,
        num_heads: int = 8,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.encoder = TransformerEncoder(
            num_layers=num_encoder_layers,
            dim_model=dim_model,
            num_heads=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.decoder = TransformerDecoder(
            num_layers=num_decoder_layers,
            dim_model=dim_model,
            num_heads=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.linear = nn.Linear(dim_model, dim_model)

    def forward(self, src: Tensor, tgt: Tensor) -> Tensor:
        memory = self.encoder(src)
        output = self.decoder(tgt, memory)
        return self.linear(output)


In [52]:
def train_transformer(
    model,
    data_loader,
    optimizer,
    criterion,
    num_epochs=1,
    device='cuda' if torch.cuda.is_available() else 'cpu'
):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        for src, tgt in data_loader:
            src, tgt = src.to(device), tgt.to(device)
            optimizer.zero_grad()
            output = model(src, tgt)
            loss = criterion(output, tgt)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

src = torch.rand((32, 10, 512))
tgt = torch.rand((32, 10, 512))
dataset = TensorDataset(src, tgt)
data_loader = DataLoader(dataset, batch_size=32)

# Instantiate the model
model = Transformer(
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_model=512,
    num_heads=8,
    dim_feedforward=2048,
    dropout=0.1
)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Train the model
train_transformer(model, data_loader, optimizer, criterion, num_epochs=1)

Epoch [1/1], Loss: 1.311914086341858


In [ ]:
class Transfomer(nn.Module):
  def __init__(self, ... ):
    super().__init__()
    pass

  def forward(self):
    pass